In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.ga import GA_VRP
from optimization.sa import SA_VRP
from optimization.tabu_search import TS_VRP
from optimization.dka import DKA_VRP
from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP
from optimization.kma import KMA_VRP
from optimization.woa import WOA_VRP
from optimization.vns import VNS_VRP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix#,tourid,idhotel

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## Outlier Sensitivity

In [5]:
hotel,tur,timematrix = generate_data(n=87,random_state=40)
dwaktu,dtarif,drating = 1,0,0

In [6]:
id_outlier = [39, 58, 62, 64, 65]

In [14]:
min_days = 1
max_days = 13

In [7]:
outlier_sensitivity_analysis = {1 : {},
                                2 : {},
                                3 : {},
                                4 : {},
                                5 : {}
                                6 : {},
                                7 : {},
                                8 : {},
                                9 : {},
                                10 : {},
                                11 : {},
                                12 : {},
                               13 : {}}

for n_days in outlier_sensitivity_analysis:
    travel_days = n_days
    
    kma_vrp = KMA_VRP(random_state=20)
    results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_kma_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["kma_vrp"] = num_outlier
    
    woa_vrp = WOA_VRP(random_state=20)
    results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_woa_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["woa_vrp"] = num_outlier
    
    vns_vrp = VNS_VRP(random_state=100)
    results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_vns_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["vns_vrp"] = num_outlier

In [11]:
for n_days in outlier_sensitivity_analysis:
    if n_days>5:
        continue
    travel_days = n_days
    
    kma_vrp = KMA_VRP(random_state=20)
    results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_kma_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["kma_vrp"] = num_outlier
    
    woa_vrp = WOA_VRP(random_state=20)
    results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_woa_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["woa_vrp"] = num_outlier
    
    vns_vrp = VNS_VRP(random_state=100)
    results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_vns_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["vns_vrp"] = num_outlier

In [12]:
outlier_sensitivity_analysis

{6: {'kma_vrp': 0, 'woa_vrp': 1, 'vns_vrp': 0},
 7: {'kma_vrp': 0, 'woa_vrp': 1, 'vns_vrp': 2},
 8: {'kma_vrp': 2, 'woa_vrp': 2, 'vns_vrp': 0},
 9: {'kma_vrp': 2, 'woa_vrp': 2, 'vns_vrp': 2},
 10: {'kma_vrp': 2, 'woa_vrp': 3, 'vns_vrp': 4},
 11: {'kma_vrp': 4, 'woa_vrp': 4, 'vns_vrp': 3},
 12: {'kma_vrp': 4, 'woa_vrp': 4, 'vns_vrp': 3},
 13: {'kma_vrp': 4, 'woa_vrp': 4, 'vns_vrp': 5},
 1: {'kma_vrp': 0, 'woa_vrp': 0, 'vns_vrp': 0},
 2: {'kma_vrp': 0, 'woa_vrp': 0, 'vns_vrp': 0},
 3: {'kma_vrp': 0, 'woa_vrp': 1, 'vns_vrp': 0},
 4: {'kma_vrp': 0, 'woa_vrp': 0, 'vns_vrp': 0},
 5: {'kma_vrp': 0, 'woa_vrp': 1, 'vns_vrp': 1}}

In [15]:
result = pd.DataFrame()
for i in outlier_sensitivity_analysis:
    if i not in range(min_days,max_days+1):
        continue
    else:
        travel_days = i
        methods = []
        num_outlier = []
        for j in outlier_sensitivity_analysis[i]:
            methods.append(j)
            num_outlier.append(outlier_sensitivity_analysis[i][j])
        temp_res = pd.DataFrame({'methods':methods,
                                'num_outlier':num_outlier})
        temp_res['travel_days'] = travel_days
        result = pd.concat([result,temp_res])

In [16]:
result

,methods,num_outlier,travel_days
0,kma_vrp,0,6
1,woa_vrp,1,6
2,vns_vrp,0,6
0,kma_vrp,0,7
1,woa_vrp,1,7
2,vns_vrp,2,7
0,kma_vrp,2,8
1,woa_vrp,2,8
2,vns_vrp,0,8
0,kma_vrp,2,9


In [17]:
result.to_excel('kma_outlier_sensitivity.xlsx',index=False)

## General Performance (50 random nodes)

In [5]:
def result_to_xlsx_rd_nodes(result,random_ls,filename):
    df = pd.DataFrame()
    for method in result:
        res_method = result[method]
        df_method = {
            "random_seed" : random_ls,
            "fitness" : [],
            "fitness_1" : [],
            "fitness_2" : [],
            "fitness_3" : [],
            "poi_included" : [],
            "rating" : [],
            "tarif" : [],
            "total_duration" : [],
            "run_time" : [],
        }
        for res in res_method:
            df_method["fitness"].append(res['fitness'])
            df_method["fitness_1"].append(res['daily_fitness'][0])
            df_method["fitness_2"].append(res['daily_fitness'][1])
            df_method["fitness_3"].append(res['daily_fitness'][2])
            df_method["poi_included"].append(res['poi_included'])
            df_method["rating"].append(res['avg_rating'])
            df_method["tarif"].append(res['total_tarif'])
            df_method["total_duration"].append(res['total_duration'])
            df_method["run_time"].append(res['run_time'])
        df_method = pd.DataFrame(df_method)
        df_method['method'] = method
        df = pd.concat([df,df_method])
    df.to_excel(filename,index=False)

### All DOI 1

In [7]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "kma_vrp": [],
    "woa_vrp": [],
    "vns_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    kma_vrp = KMA_VRP(random_state=20)
    results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["kma_vrp"].append(results_kma_vrp)
    
    woa_vrp = WOA_VRP(random_state=20)
    results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["woa_vrp"].append(results_woa_vrp)
    
    vns_vrp = VNS_VRP(random_state=100)
    results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["vns_vrp"].append(results_vns_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [8]:
random_node_analysis

{'kma_vrp': [{'solution': [[4, 99, 9, 22, 66, 93, 15, 52],
    [10, 50, 90, 14, 88, 26, 7, 74],
    [55, 65, 35, 51, 2]],
   'fitness': 0.6721267595088348,
   'daily_fitness': [0.6586936582809223,
    0.6955669112508734,
    0.6222331411600278],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.609523809523808,
   'total_tarif': 47000,
   'total_duration': 126720,
   'run_time': 2.358727216720581},
  {'solution': [[49, 89, 22, 36, 51, 74],
    [32, 57, 18, 12, 21, 66, 46, 2],
    [34, 99, 6, 44, 10]],
   'fitness': 0.6433238023771327,
   'daily_fitness': [0.5926683874139627,
    0.6415580957063258,
    0.6044215011471648],
   'days_needed': 3,
   'poi_included': 19,
   'avg_rating': 4.5736842105263165,
   'total_tarif': 23500,
   'total_duration': 116280,
   'run_time': 2.350503921508789},
  {'solution': [[32, 8, 81, 39, 36, 2, 46, 15],
    [20, 74, 93, 71, 3],
    [33, 22, 85, 68, 50, 57, 43]],
   'fitness': 0.6103031259121893,
   'daily_fitness': [0.5994622068422484,
   

In [9]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"kma_random_nodes_alldoi_1.xlsx")

### Travel Duration 1, other 0

In [10]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "kma_vrp": [],
    "woa_vrp": [],
    "vns_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i])
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("random state = ",random_ls[i])
    
    kma_vrp = KMA_VRP(random_state=20)
    results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["kma_vrp"].append(results_kma_vrp)
    
    woa_vrp = WOA_VRP(random_state=20)
    results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["woa_vrp"].append(results_woa_vrp)
    
    vns_vrp = VNS_VRP(random_state=100)
    results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["vns_vrp"].append(results_vns_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [12]:
random_node_analysis

{'kma_vrp': [{'solution': [[52, 72, 2, 15, 66, 90, 10, 88, 35, 4],
    [74, 31, 48, 14, 7, 50, 9, 22],
    [99, 55, 38, 64, 83]],
   'fitness': 0.6419367283950618,
   'daily_fitness': [0.49675925925925923,
    0.47106481481481477,
    0.4337962962962963],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.578260869565217,
   'total_tarif': 205000,
   'total_duration': 124860,
   'run_time': 5.329279899597168},
  {'solution': [[80, 22, 67, 44, 18, 89, 2, 32],
    [46, 12, 66, 34, 51, 21, 36, 10],
    [74, 97, 48, 49, 99, 5, 57]],
   'fitness': 0.6423611111111112,
   'daily_fitness': [0.467824074074074,
    0.4668981481481481,
    0.4666666666666667],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.530434782608695,
   'total_tarif': 195500,
   'total_duration': 124920,
   'run_time': 5.51571249961853},
  {'solution': [[81, 50, 48, 74, 57, 22, 2, 32, 15, 3],
    [29, 38, 47, 46, 36, 33],
    [8, 99, 20, 85, 53, 5, 43]],
   'fitness': 0.6339506172839506,
   'dail

In [15]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"kma_random_nodes_dwaktu_1.xlsx")

### Tarif 1, other 0

In [16]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "kma_vrp": [],
    "woa_vrp": [],
    "vns_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("random state = ",random_ls[i])
    
    kma_vrp = KMA_VRP(random_state=20)
    results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["kma_vrp"].append(results_kma_vrp)
    
    woa_vrp = WOA_VRP(random_state=20)
    results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["woa_vrp"].append(results_woa_vrp)
    
    vns_vrp = VNS_VRP(random_state=100)
    results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["vns_vrp"].append(results_vns_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [17]:
random_node_analysis

{'kma_vrp': [{'solution': [[10, 74, 7, 50, 14, 66, 26, 2, 52],
    [55, 34, 15, 22, 9, 93, 88],
    [90, 38, 4, 51, 35]],
   'fitness': 0.8158623049149778,
   'daily_fitness': [0.65, 0.6025157232704402, 0.5314465408805031],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.604761904761904,
   'total_tarif': 35000,
   'total_duration': 130680,
   'run_time': 2.382383108139038},
  {'solution': [[57, 38, 99, 71, 49, 36, 21],
    [44, 12, 46, 6, 18, 10, 32, 2],
    [89, 74, 51, 34, 66, 22]],
   'fitness': 0.8176991150442477,
   'daily_fitness': [0.5759587020648967,
    0.6227138643067847,
    0.5867256637168141],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.561904761904762,
   'total_tarif': 36500,
   'total_duration': 127920,
   'run_time': 5.8664679527282715},
  {'solution': [[81, 22, 32, 8, 15, 20, 71],
    [38, 57, 2, 42, 43, 3, 36],
    [50, 68, 99, 53, 74, 46, 85]],
   'fitness': 0.7797635605006954,
   'daily_fitness': [0.591631896152063,
    0.57772369

In [18]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"kma_random_nodes_dtarif_1.xlsx")

### Rating 1, other 0

In [19]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "kma_vrp": [],
    "woa_vrp": [],
    "vns_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("random state = ",random_ls[i])
    
    kma_vrp = KMA_VRP(random_state=20)
    results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["kma_vrp"].append(results_kma_vrp)
    
    woa_vrp = WOA_VRP(random_state=20)
    results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["woa_vrp"].append(results_woa_vrp)
    
    vns_vrp = VNS_VRP(random_state=100)
    results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["vns_vrp"].append(results_vns_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [20]:
random_node_analysis

{'kma_vrp': [{'solution': [[9, 4, 88, 48, 10, 7, 74, 26, 66],
    [55, 2, 90, 52, 14, 35, 15, 50, 93],
    [72, 99, 64, 34, 22, 83]],
   'fitness': 0.8420524691358025,
   'daily_fitness': [0.5635802469135806,
    0.5450617283950616,
    0.42407407407407416],
   'days_needed': 3,
   'poi_included': 24,
   'avg_rating': 4.591666666666667,
   'total_tarif': 179500,
   'total_duration': 128940,
   'run_time': 11.82421612739563},
  {'solution': [[57, 12, 67, 44, 21, 74, 64, 66],
    [89, 22, 71, 18, 46, 32, 36],
    [10, 99, 48, 39, 97, 2, 49]],
   'fitness': 0.756246492704826,
   'daily_fitness': [0.4395833333333332,
    0.42380952380952364,
    0.3380952380952382],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.559090909090909,
   'total_tarif': 164000,
   'total_duration': 127320,
   'run_time': 10.437416791915894},
  {'solution': [[74, 57, 81, 3, 48, 85, 53, 32, 50],
    [8, 39, 20, 99, 22, 2, 36],
    [47, 46, 68, 15, 33, 5, 43]],
   'fitness': 0.7451345755693578,
   'd

In [21]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"kma_random_nodes_drating_1.xlsx")

## Random DOI Combinations

In [22]:
def result_to_xlsx_rd_doi(result,random_doi,random_ls,filename):
    df_doi = pd.DataFrame()
    for doi_res in range(len(result)):
        df = pd.DataFrame()
        for method in result[doi_res]:
            res_method = result[doi_res][method]
            df_method = {
                "random_seed" : random_ls,
                "fitness" : [],
                "fitness_1" : [],
                "fitness_2" : [],
                "fitness_3" : [],
                "poi_included" : [],
                "rating" : [],
                "tarif" : [],
                "total_duration" : [],
                "run_time" : [],
            }
            for res in res_method:
                df_method["fitness"].append(res['fitness'])
                df_method["fitness_1"].append(res['daily_fitness'][0])
                df_method["fitness_2"].append(res['daily_fitness'][1])
                df_method["fitness_3"].append(res['daily_fitness'][2])
                df_method["poi_included"].append(res['poi_included'])
                df_method["rating"].append(res['avg_rating'])
                df_method["tarif"].append(res['total_tarif'])
                df_method["total_duration"].append(res['total_duration'])
                df_method["run_time"].append(res['run_time'])
            df_method = pd.DataFrame(df_method)
            df_method['method'] = method
            df = pd.concat([df,df_method])
        
        doi = random_doi[doi_res]
        df['dwaktu'] = doi[0]
        df['dtarif'] = doi[1]
        df['drating'] = doi[2]
        df = df[['dwaktu','dtarif','drating','method','random_seed','fitness','fitness_1','fitness_2','fitness_3',
                'poi_included','rating','tarif','total_duration','run_time']]
        df_doi = pd.concat([df_doi,df])
#     return df_doi
    df_doi.to_excel(filename,index=False)

In [23]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [24]:
doi_combi

[(0.15, 0.45, 0.77),
 (0.71, 0.73, 0.43),
 (0.8, 0.53, 0.08),
 (0.46, 0.05, 0.93),
 (0.95, 0.34, 0.31),
 (0.77, 0.2, 0.18),
 (0.19, 0.35, 0.63),
 (0.96, 0.21, 0.96),
 (0.56, 0.9, 0.82),
 (0.16, 0.65, 0.12)]

In [25]:
doi_combi_analysis = []
for doi in doi_combi:
    print("doi combination = ",doi)
    random_ls = list(range(5,(5*50)+1,5))
    random_node_analysis ={
        "kma_vrp": [],
        "woa_vrp": [],
        "vns_vrp": [],
    }
    for i in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
        travel_days = 3
        dwaktu,dtarif,drating = doi

        print("random state = ",random_ls[i])

        kma_vrp = KMA_VRP(random_state=20)
        results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["kma_vrp"].append(results_kma_vrp)

        woa_vrp = WOA_VRP(random_state=20)
        results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["woa_vrp"].append(results_woa_vrp)

        vns_vrp = VNS_VRP(random_state=100)
        results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["vns_vrp"].append(results_vns_vrp)
    
    doi_combi_analysis.append(random_node_analysis)

doi combination =  (0.15, 0.45, 0.77)
random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
ra

random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250
doi combination =  (0.16, 0.65, 0.12)


In [26]:
doi_combi_analysis

[{'kma_vrp': [{'solution': [[2, 9, 4, 35, 10, 7, 50, 26, 22],
     [65, 93, 14, 88, 74, 66],
     [99, 15, 52, 90, 55, 34, 51]],
    'fitness': 0.7863774698953809,
    'daily_fitness': [0.620576279115939,
     0.5729022002258609,
     0.5391439773903356],
    'days_needed': 3,
    'poi_included': 22,
    'avg_rating': 4.604545454545455,
    'total_tarif': 47000,
    'total_duration': 129000,
    'run_time': 8.818528413772583},
   {'solution': [[74, 89, 10, 18, 46, 22, 66, 2],
     [44, 99, 12, 39, 48, 21, 32, 36],
     [64, 51, 34, 57, 49]],
    'fitness': 0.7253265425973554,
    'daily_fitness': [0.564504219409283,
     0.49507839305643714,
     0.42467279870886887],
    'days_needed': 3,
    'poi_included': 21,
    'avg_rating': 4.576190476190477,
    'total_tarif': 81500,
    'total_duration': 124560,
    'run_time': 3.939138412475586},
   {'solution': [[39, 46, 81, 48, 20, 2, 15, 3, 50],
     [68, 71, 33, 53, 32, 36],
     [8, 22, 57, 74, 93, 43]],
    'fitness': 0.7094582776165088

In [27]:
result_to_xlsx_rd_doi(doi_combi_analysis,doi_combi,random_ls,"kma_random_doi_combi.xlsx")

## 87 POIs

In [28]:
def result_to_xlsx_all_poi(result,n,random_ls,filename):
    df_result = pd.DataFrame()
    for i in result:
        for j in range(len(result[i])):
            temp = pd.DataFrame(result[i][j],columns=["fitness","run_time"])
            temp['n'] = n[j]
            temp['method'] = i
            temp['random_seed'] = random_ls
            df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [29]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "kma_vrp": [],
    "woa_vrp": [],
    "vns_vrp": [],
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(5,(5*50)+1,5))
    random_node = {
        "kma_vrp": [],
        "woa_vrp": [],
        "vns_vrp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        kma_vrp = KMA_VRP(random_state=20)
        results_kma_vrp = get_analysis(kma_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["kma_vrp"].append(results_kma_vrp)

        woa_vrp = WOA_VRP(random_state=20)
        results_woa_vrp = get_analysis(woa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["woa_vrp"].append(results_woa_vrp)

        vns_vrp = VNS_VRP(random_state=100)
        results_vns_vrp = get_analysis(vns_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["vns_vrp"].append(results_vns_vrp)
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  10
rando

random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  50
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state

random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250


In [30]:
run_time_analysis

{'kma_vrp': [[{'solution': [[93, 48, 88, 35, 99]],
    'fitness': 0.7043981481481482,
    'daily_fitness': [0.5565972222222222],
    'days_needed': 1,
    'poi_included': 5,
    'avg_rating': 4.58,
    'total_tarif': 52500,
    'total_duration': 28980,
    'run_time': 1.7190513610839844},
   {'solution': [[58, 66, 80, 5, 2]],
    'fitness': 0.7103086419753084,
    'daily_fitness': [0.5654629629629626],
    'days_needed': 1,
    'poi_included': 5,
    'avg_rating': 4.4799999999999995,
    'total_tarif': 70000,
    'total_duration': 38640,
    'run_time': 1.714184045791626},
   {'solution': [[2, 71, 29, 22, 5]],
    'fitness': 0.7047839506172839,
    'daily_fitness': [0.5571759259259259],
    'days_needed': 1,
    'poi_included': 5,
    'avg_rating': 4.5,
    'total_tarif': 70000,
    'total_duration': 35160,
    'run_time': 1.6789729595184326},
   {'solution': [[21, 96, 36, 90, 97]],
    'fitness': 0.7191358024691361,
    'daily_fitness': [0.5787037037037043],
    'days_needed': 1,
    

In [31]:
result_to_xlsx_all_poi(run_time_analysis,n,random_ls,"kma_various_numbers_pois_all_1.xlsx")

# Uji Statistik

In [77]:
def uji_statistik(first_data,second_data,alternative="two-sided",first_data_name="first",second_data_name="second"):
    # first_data is the target data, e.g. the results of Hybrid ACS-BSO 
    # second_data is a list of results from other method
    # the test is using Wilcoxon Rank-Sum Test
    p_values = []
    second_data_name = ["second" for i in range(len(second_data))] if second_data_name == "second" else second_data_name
    for i in range(len(second_data)):
        statistic, p_value = stats.ranksums(first_data, second_data[i],alternative=alternative)
        p_values.append(p_value)
        print(f"Wilcoxon Rank-Sum Test ({first_data_name} vs. {second_data_name[i]}):")
        print("Test Statistic:", statistic)
        print("p-value:", p_value)
        print("")
    return p_values

def save_to_excel_sheet(df,filename,sheet_name):
    try:
        with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:

            # use to_excel function and specify the sheet_name and index to 
            # store the dataframe in specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    except:
        with pd.ExcelWriter(filename, mode="w", engine="openpyxl") as writer:

            # use to_excel function and specify the sheet_name and index to 
            # store the dataframe in specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)

## General performance

### All DOI 1

In [2]:
results_df = pd.read_excel("dka_random_nodes_alldoi_1.xlsx")
results_df2 = pd.read_excel("kma_random_nodes_alldoi_1.xlsx")
results_df = pd.concat([results_df,results_df2])
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.681458,0.709375,0.625561,0.650876,21,4.614286,37000,125100,4.769661,acs_vrp
1,10,0.644172,0.631843,0.602275,0.602765,20,4.580000,24500,124620,3.170985,acs_vrp
2,15,0.611977,0.603838,0.579350,0.546002,21,4.633333,81500,126480,2.801009,acs_vrp
3,20,0.678256,0.665319,0.629272,0.618132,22,4.590909,41000,122520,3.208303,acs_vrp
4,25,0.628851,0.631071,0.607775,0.587981,20,4.560000,86000,121860,5.879171,acs_vrp


In [60]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
1,acs_vrp,0.659326,3.746193
6,sa_vrp,0.658175,0.971710
5,kma_vrp,0.656306,5.064276
8,vns_vrp,0.642761,0.167079
3,dka_vrp,0.640759,0.462857
9,woa_vrp,0.640459,0.414673
7,ts_vrp,0.640236,0.665049
0,aco_vrp,0.639912,2.691196
4,ga_vrp,0.630925,0.742191
2,bso_vrp,0.552943,0.378404


In [61]:
metrics = ["fitness","run_time"]

df_stat = pd.DataFrame()
for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "two-sided"
    first_data = results_df[results_df['method']=="kma_vrp"][metrics].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["kma_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'metrics':metric,
                                               'method':second_data_name,
                                               'p_value':p_values})])

metric:  fitness
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: -0.6962758074211683
p-value: 0.48625612084642644

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 3.157369502959358
p-value: 0.001591994751289423

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 8.582805744944105
p-value: 9.25866337958132e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 4.722266614688122
p-value: 2.332306574370423e-06

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 3.233201521589386
p-value: 0.0012241117846865238

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: -0.5032488509083692
p-value: 0.6147893229828156

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 3.260776801091214
p-value: 0.0011110746216913378

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 3.315927360094871
p-value: 0.0009133956701454181

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Statistic: 2.392155496783618
p-value: 0.01

In [62]:
df_stat

,metrics,method,p_value
0,fitness,acs_vrp,4.862561e-01
1,fitness,aco_vrp,1.591995e-03
2,fitness,bso_vrp,9.258663e-18
3,fitness,ga_vrp,2.332307e-06
4,fitness,ts_vrp,1.224112e-03
5,fitness,sa_vrp,6.147893e-01
6,fitness,dka_vrp,1.111075e-03
7,fitness,woa_vrp,9.133957e-04
8,fitness,vns_vrp,1.674974e-02
0,run_time,acs_vrp,5.019422e-03


In [78]:
save_to_excel_sheet(grouped.sort_values('fitness',ascending=False),filename="kma_experiment_results.xlsx",sheet_name="alldoi_1_results")
save_to_excel_sheet(df_stat,filename="kma_experiment_results.xlsx",sheet_name="alldoi_1_p_value")

### Travel Duration 1

In [79]:
results_df = pd.read_excel("dka_random_nodes_dwaktu_1.xlsx")
results_df2 = pd.read_excel("kma_random_nodes_dwaktu_1.xlsx")
results_df = pd.concat([results_df,results_df2])
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.648032,0.510880,0.424074,0.477778,23,4.556522,104500,121980,3.011678,acs_vrp
1,10,0.642361,0.487963,0.472685,0.440741,23,4.534783,196000,124920,4.785303,acs_vrp
2,15,0.631211,0.481481,0.438426,0.463657,23,4.604348,260500,129540,3.239691,acs_vrp
3,20,0.663850,0.483102,0.487037,0.457639,24,4.558333,283000,122400,3.555043,acs_vrp
4,25,0.626852,0.468287,0.459491,0.459259,22,4.500000,205500,124320,7.371741,acs_vrp


In [80]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
1,acs_vrp,0.653629,3.864807
5,kma_vrp,0.652954,6.356812
0,aco_vrp,0.652414,2.674487
6,sa_vrp,0.651903,0.946983
7,ts_vrp,0.641319,0.648231
8,vns_vrp,0.640123,0.164907
3,dka_vrp,0.637856,0.436884
9,woa_vrp,0.634821,0.430723
4,ga_vrp,0.629941,0.713939
2,bso_vrp,0.585955,0.342127


In [81]:
metrics = ["fitness","run_time"]

df_stat = pd.DataFrame()
for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "two-sided"
    first_data = results_df[results_df['method']=="kma_vrp"][metrics].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["kma_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'metrics':metric,
                                               'method':second_data_name,
                                               'p_value':p_values})])

metric:  fitness
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: -0.26541206520509886
p-value: 0.7906920426705077

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 0.017234549688642783
p-value: 0.9862494996122527

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 7.769334999640166
p-value: 7.889924586630394e-15

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 4.832567732695436
p-value: 1.3478323275976263e-06

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 2.4059431365345323
p-value: 0.016130775289776193

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 0.11719493788277091
p-value: 0.9067055782618836

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 3.2228607917762004
p-value: 0.001269172182826938

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 3.877773679944626
p-value: 0.00010541671182993176

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Statistic: 2.5610540837323175
p-value:

In [82]:
df_stat

,metrics,method,p_value
0,fitness,acs_vrp,7.906920e-01
1,fitness,aco_vrp,9.862495e-01
2,fitness,bso_vrp,7.889925e-15
3,fitness,ga_vrp,1.347832e-06
4,fitness,ts_vrp,1.613078e-02
5,fitness,sa_vrp,9.067056e-01
6,fitness,dka_vrp,1.269172e-03
7,fitness,woa_vrp,1.054167e-04
8,fitness,vns_vrp,1.043551e-02
0,run_time,acs_vrp,7.861201e-08


In [83]:
save_to_excel_sheet(grouped.sort_values('fitness',ascending=False),filename="kma_experiment_results.xlsx",sheet_name="dwaktu_1_results")
save_to_excel_sheet(df_stat,filename="kma_experiment_results.xlsx",sheet_name="dwaktu_1_p_value")

### Tarif 1

In [84]:
results_df = pd.read_excel("dka_random_nodes_dtarif_1.xlsx")
results_df2 = pd.read_excel("kma_random_nodes_dtarif_1.xlsx")
results_df = pd.concat([results_df,results_df2])
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.809434,0.593082,0.592138,0.583648,21,4.604762,43000,128340,1.987593,acs_vrp
1,10,0.807227,0.575221,0.607817,0.598083,20,4.565000,29500,127380,4.700382,acs_vrp
2,15,0.791200,0.622184,0.550649,0.561034,21,4.623810,83500,129600,3.271467,acs_vrp
3,20,0.841992,0.639924,0.627036,0.550588,22,4.586364,39000,127800,5.548777,acs_vrp
4,25,0.780875,0.588576,0.555925,0.569949,21,4.528571,106000,126180,3.074423,acs_vrp


In [85]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
5,kma_vrp,0.826179,4.624728
6,sa_vrp,0.825749,0.957705
1,acs_vrp,0.825103,3.643509
7,ts_vrp,0.807096,0.663548
9,woa_vrp,0.803911,0.365912
3,dka_vrp,0.799982,0.406919
8,vns_vrp,0.795416,0.117696
4,ga_vrp,0.794564,0.767411
0,aco_vrp,0.786884,2.803389
2,bso_vrp,0.661432,0.193773


In [86]:
metrics = ["fitness","run_time"]

df_stat = pd.DataFrame()
for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "two-sided"
    first_data = results_df[results_df['method']=="kma_vrp"][metrics].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["kma_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'metrics':metric,
                                               'method':second_data_name,
                                               'p_value':p_values})])

metric:  fitness
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: 0.3481379037105842
p-value: 0.7277366196432564

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 6.3078451860432585
p-value: 2.829470201315512e-10

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 5.4943744407393185
p-value: 3.920977732925409e-08

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 4.025990807266954
p-value: 5.6735905386563855e-05

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 0.13098257763368515
p-value: 0.8957890896901726

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 4.5774963973035225
p-value: 4.7057389178179324e-06

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 4.356894161288896
p-value: 1.3192101810016442e-05

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Statistic: 5.066957608460978
p-value

In [87]:
save_to_excel_sheet(grouped.sort_values('fitness',ascending=False),filename="kma_experiment_results.xlsx",sheet_name="dtarif_1_results")
save_to_excel_sheet(df_stat,filename="kma_experiment_results.xlsx",sheet_name="dtarif_1_p_value")

### Rating 1

In [88]:
results_df = pd.read_excel("dka_random_nodes_drating_1.xlsx")
results_df2 = pd.read_excel("kma_random_nodes_drating_1.xlsx")
results_df = pd.concat([results_df,results_df2])
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.826339,0.594444,0.498148,0.445833,23,4.595652,179500,128040,2.198029,acs_vrp
1,10,0.774953,0.477083,0.402381,0.358333,23,4.569565,142000,129420,5.177960,acs_vrp
2,15,0.738227,0.454762,0.320748,0.310544,22,4.640909,151500,126180,3.272198,acs_vrp
3,20,0.795721,0.550000,0.427778,0.377778,23,4.591304,235000,127980,1.410371,acs_vrp
4,25,0.750396,0.483333,0.395455,0.365152,21,4.566667,139000,127080,2.331297,acs_vrp


In [89]:
grouped = results_df.groupby('method').agg({'fitness':'mean',
                                            'run_time':'mean'}).reset_index()
grouped.sort_values('fitness',ascending=False)

,method,fitness,run_time
5,kma_vrp,0.799579,5.528577
1,acs_vrp,0.796156,3.169225
6,sa_vrp,0.793384,0.963424
7,ts_vrp,0.785960,0.673597
9,woa_vrp,0.782584,0.398832
3,dka_vrp,0.780811,0.407809
8,vns_vrp,0.779596,0.123323
0,aco_vrp,0.775737,2.412886
4,ga_vrp,0.775300,0.661126
2,bso_vrp,0.708324,0.226886


In [90]:
metrics = ["fitness","run_time"]

df_stat = pd.DataFrame()
for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "two-sided"
    first_data = results_df[results_df['method']=="kma_vrp"][metrics].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["kma_vrp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'metrics':metric,
                                               'method':second_data_name,
                                               'p_value':p_values})])

metric:  fitness
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: 0.5652932297874832
p-value: 0.5718743328719997

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 3.315927360094871
p-value: 0.0009133956701454181

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 7.638352422006481
p-value: 2.2001894659547157e-14

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 3.3366088197212425
p-value: 0.0008480722124719495

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 1.9371633850034486
p-value: 0.05272537448153168

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 1.0064977018167385
p-value: 0.3141762165936888

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 2.7851032296846734
p-value: 0.00535106842508092

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 2.4197307762854465
p-value: 0.015532001698122919

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Statistic: 2.833359968812873
p-value: 0.004

In [91]:
save_to_excel_sheet(grouped.sort_values('fitness',ascending=False),filename="kma_experiment_results.xlsx",sheet_name="drating_1_results")
save_to_excel_sheet(df_stat,filename="kma_experiment_results.xlsx",sheet_name="drating_1_p_value")

## DOI Combination

In [123]:
results_df = pd.read_excel("dka_random_doi_combi.xlsx")
results_df2 = pd.read_excel("kma_random_doi_combi.xlsx")
results_df = pd.concat([results_df,results_df2])
results_df.head()

,dwaktu,dtarif,drating,method,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
0,0.15,0.45,0.77,acs_vrp,5,0.788916,0.554363,0.536150,0.613935,23,4.591304,75000,128280,5.805615
1,0.15,0.45,0.77,acs_vrp,10,0.736776,0.571812,0.449716,0.491406,21,4.580952,52000,125100,4.605406
2,0.15,0.45,0.77,acs_vrp,15,0.714983,0.475079,0.449430,0.475128,22,4.627273,110500,133320,3.156808
3,0.15,0.45,0.77,acs_vrp,20,0.783644,0.601456,0.538273,0.512053,23,4.595652,57000,129420,4.294260
4,0.15,0.45,0.77,acs_vrp,25,0.730412,0.547887,0.491160,0.474618,22,4.540909,146000,131040,4.416386


In [124]:
grouped = results_df.groupby(['dwaktu','dtarif','drating','method']).agg({'fitness':'mean',
                                                                          'run_time':'mean'})

grouped

fitness  run_time
dwaktu dtarif drating method                     
0.15   0.45   0.77    aco_vrp  0.746655  3.153367
                      acs_vrp  0.763987  4.320484
                      bso_vrp  0.672562  0.401293
                      dka_vrp  0.751793  0.497686
                      ga_vrp   0.741615  0.738393
...                                 ...       ...
0.96   0.21   0.96    kma_vrp  0.645501  5.460583
                      sa_vrp   0.643477  0.942659
                      ts_vrp   0.631328  0.653747
                      vns_vrp  0.632843  0.170804
                      woa_vrp  0.628678  0.443154

[100 rows x 2 columns]

In [125]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [126]:
df_stat = pd.DataFrame()
for doi in doi_combi:
    print("doi combination : ",doi)
    metrics = "fitness"
    print("metric: ",metrics)
    alternative = "two-sided"
    first_data = results_df[(results_df['method']=="kma_vrp")&
                        (results_df['dwaktu']==doi[0])&
                        (results_df['dtarif']==doi[1])&
                        (results_df['drating']==doi[2])][metrics].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["kma_vrp"])]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&
                        (results_df['dwaktu']==doi[0])&
                        (results_df['dtarif']==doi[1])&
                        (results_df['drating']==doi[2])][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'dwaktu':doi[0],
                                               'dtarif':doi[1],
                                               'drating':doi[2],
                                               'metrics':metrics,
                                               'method':second_data_name,
                                               'p_value':p_values})])

doi combination :  (0.15, 0.45, 0.77)
metric:  fitness
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: -0.04825673912819979
p-value: 0.9615116315341786

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 3.322821179970328
p-value: 0.0008911201943750174

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 8.09334453378665
p-value: 5.804837098308177e-16

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 3.7295565526222982
p-value: 0.00019181705280887598

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 2.3507925775308753
p-value: 0.018733473489472924

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 0.22749605589008473
p-value: 0.8200380357884649

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 2.130190341516248
p-value: 0.03315590213557952

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 2.9505549066956442
p-value: 0.0031720365921230715

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test St

In [127]:
df_stat2 = pd.DataFrame()
for doi in doi_combi:
    print("doi combination : ",doi)
    metrics = "run_time"
    print("metric: ",metrics)
    alternative = "two-sided"
    first_data = results_df[(results_df['method']=="kma_vrp")&
                        (results_df['dwaktu']==doi[0])&
                        (results_df['dtarif']==doi[1])&
                        (results_df['drating']==doi[2])][metrics].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["kma_vrp"])]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&
                        (results_df['dwaktu']==doi[0])&
                        (results_df['dtarif']==doi[1])&
                        (results_df['drating']==doi[2])][metrics].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat2 = pd.concat([df_stat2,pd.DataFrame({'dwaktu':doi[0],
                                               'dtarif':doi[1],
                                               'drating':doi[2],
                                               'metrics':metrics,
                                               'method':second_data_name,
                                               'p_value':p_values})])

doi combination :  (0.15, 0.45, 0.77)
metric:  run_time
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: 1.0202853415676527
p-value: 0.307593153232919

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 4.081141366270611
p-value: 4.481508612303341e-05

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Sta

In [128]:
df_stat = df_stat.pivot(index=['metrics','dwaktu','dtarif','drating'],columns='method',values='p_value').reset_index()
df_stat2 = df_stat2.pivot(index=['metrics','dwaktu','dtarif','drating'],columns='method',values='p_value').reset_index()
df_stat = pd.concat([df_stat,df_stat2])

In [129]:
grouped1 = grouped.reset_index().copy()
grouped1['metrics'] = 'fitness'
grouped1 = grouped1.pivot(index=['metrics','dwaktu','dtarif','drating'],columns='method',values='fitness').reset_index()

grouped2 = grouped.reset_index().copy()
grouped2['metrics'] = 'run_time'
grouped2 = grouped2.pivot(index=['metrics','dwaktu','dtarif','drating'],columns='method',values='run_time').reset_index()

grouped = pd.concat([grouped1,grouped2])

In [130]:
save_to_excel_sheet(grouped,filename="kma_experiment_results.xlsx",sheet_name="doi_combi_results")
save_to_excel_sheet(df_stat,filename="kma_experiment_results.xlsx",sheet_name="doi_combi_p_value")

## All POI

In [140]:
results_df1 = pd.read_excel("dka_various_numbers_pois_all_1.xlsx")
results_df2 = pd.read_excel("dka_various_numbers_pois_all_1_v2.xlsx")
results_df3 = pd.read_excel("kma_various_numbers_pois_all_1.xlsx")
results_df = pd.concat([results_df1,results_df2,results_df3])
results_df = results_df.reset_index(drop=True)
results_df.head()

,fitness,run_time,n,method,random_seed
0,0.704398,0.170605,5,acs_vrp,5
1,0.710309,0.173731,5,acs_vrp,10
2,0.704784,0.134615,5,acs_vrp,15
3,0.719136,0.110273,5,acs_vrp,20
4,0.715586,0.118830,5,acs_vrp,25


In [141]:
grouped = results_df.groupby(['method','n']).mean().copy()

In [142]:
n = results_df['n'].unique()
n.sort()
metric = "fitness"
alternative = "two-sided"

df_stat = pd.DataFrame()
for i in n:
    print("n: ",i)
    first_data = results_df[(results_df['method']=="kma_vrp")&(results_df['n']==i)][metric].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[results_df['method'] != "kma_vrp"]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&(results_df['n']==i)][metric].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat = pd.concat([df_stat,pd.DataFrame({'n':i,
                                               'metrics':metric,
                                               'method':second_data_name,
                                               'p_value':p_values})])

n:  5
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: 0.017234549688642783
p-value: 0.9862494996122527

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 0.2309429658278133
p-value: 0.8173591109195852

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 0.6514659782306972
p-value: 0.5147457320739065

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 0.04136291925274268
p-value: 0.9670065736331777

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 0.0
p-value: 1.0

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 0.15511094719778504
p-value: 0.8767338520233376

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 0.0
p-value: 1.0

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 0.04825673912819979
p-value: 0.9615116315341786

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Statistic: 0.2309429658278133
p-value: 0.8173591109195852

n:  10
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Stat

In [143]:
n = results_df['n'].unique()
n.sort()
metric = "run_time"
alternative = "two-sided"

df_stat2 = pd.DataFrame()
for i in n:
    print("n: ",i)
    first_data = results_df[(results_df['method']=="kma_vrp")&(results_df['n']==i)][metric].values
    first_data_name = "kma_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[results_df['method'] != "kma_vrp"]['method'].unique():
        second_data.append(results_df[(results_df['method']==method)&(results_df['n']==i)][metric].values)
        second_data_name.append(method)

    p_values = uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    df_stat2 = pd.concat([df_stat2,pd.DataFrame({'n':i,
                                               'metrics':metric,
                                               'method':second_data_name,
                                               'p_value':p_values})])

n:  5
Wilcoxon Rank-Sum Test (kma_vrp vs. acs_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. aco_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. bso_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. ga_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. ts_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. sa_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. dka_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. woa_vrp):
Test Statistic: 8.617274844321392
p-value: 6.856641447475644e-18

Wilcoxon Rank-Sum Test (kma_vrp vs. vns_vrp):
Test Statistic: 8.617274844321392
p-value: 6.8566414474

In [144]:
df_stat = df_stat.pivot(index=['metrics','n'],columns='method',values='p_value').reset_index()
df_stat2 = df_stat2.pivot(index=['metrics','n'],columns='method',values='p_value').reset_index()
df_stat = pd.concat([df_stat,df_stat2])

In [146]:
grouped1 = grouped.reset_index().copy()
grouped1['metrics'] = 'fitness'
grouped1 = grouped1.pivot(index=['metrics','n'],columns='method',values='fitness').reset_index()

grouped2 = grouped.reset_index().copy()
grouped2['metrics'] = 'run_time'
grouped2 = grouped2.pivot(index=['metrics','n'],columns='method',values='run_time').reset_index()

grouped = pd.concat([grouped1,grouped2])

In [148]:
save_to_excel_sheet(grouped,filename="kma_experiment_results.xlsx",sheet_name="various_n_pois_results")
save_to_excel_sheet(df_stat,filename="kma_experiment_results.xlsx",sheet_name="various_n_pois_p_value")